In [1]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_predict
import seaborn as sns
sns.set_style('whitegrid')
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score,accuracy_score

In [2]:
df=pd.read_excel('test.xlsx')
df.head()
df

,患者年龄,是否绝经,迈瑞CA125,迈瑞HE4,迈瑞CA199,迈瑞AFP,迈瑞NSE,迈瑞CEA,迈瑞IGF-1,迈瑞CA153,迈瑞CYFRA 21-1,良恶
0,23,0,712.66,188.21,20.29,2.77,20.15,0.85,24.37,10.56,18.23,1
1,49,0,437.61,386.24,9.23,2.73,25.52,2.65,29.37,29.32,8.95,1
2,31,0,55.73,37.53,1.00,80.65,13.65,1.27,132.43,13.39,1.54,1
3,46,0,592.89,453.94,9.88,2.84,15.98,0.45,61.16,12.18,21.98,1
4,49,0,392.84,1500.00,1.00,4.14,16.76,1.64,52.47,58.86,4.01,1
...,...,...,...,...,...,...,...,...,...,...,...,...
377,54,1,5.58,41.31,12.50,2.10,14.64,0.50,96.78,15.10,2.56,0
378,56,1,5387.80,1500.00,80.47,3.23,29.62,1.23,36.58,31.90,18.59,1
379,53,1,10.54,38.63,40.17,2.47,14.73,0.86,154.47,15.22,2.05,0
380,68,1,172.85,218.26,250.19,2.73,14.46,2.47,130.57,36.97,6.07,1


In [3]:
#分离数据
dx=df.columns[:-1]
dy=df.columns[-1]
x=df[dx]
y=df[dy]
#z-score标准化
x=(x-x.mean())/(x.std())
x

,患者年龄,是否绝经,迈瑞CA125,迈瑞HE4,迈瑞CA199,迈瑞AFP,迈瑞NSE,迈瑞CEA,迈瑞IGF-1,迈瑞CA153,迈瑞CYFRA 21-1
0,-1.123404,-0.586658,0.447869,0.152925,-0.161970,-0.090560,0.614312,-0.225344,-1.966062,-0.231347,1.944419
1,0.682041,-0.586658,0.139876,0.837184,-0.248812,-0.090934,1.151218,0.299703,-1.885958,0.071807,0.682521
2,-0.567882,-0.586658,-0.287742,-0.367725,-0.313434,0.637914,-0.035575,-0.102833,-0.234867,-0.185615,-0.325093
3,0.473720,-0.586658,0.313754,1.071110,-0.243709,-0.089905,0.197384,-0.342021,-1.376661,-0.205168,2.454345
4,0.682041,-0.586658,0.089744,4.685593,-0.313434,-0.077745,0.275371,0.005093,-1.515881,0.549162,0.010779
...,...,...,...,...,...,...,...,...,...,...,...
377,1.029241,1.700110,-0.343898,-0.354663,-0.223137,-0.096827,0.063408,-0.327437,-0.806004,-0.157982,-0.186393
378,1.168122,1.700110,5.682950,4.685593,0.310559,-0.086257,1.561147,-0.114501,-1.770449,0.113499,1.993371
379,0.959801,1.700110,-0.338344,-0.363924,-0.005874,-0.093366,0.072406,-0.222427,0.118228,-0.156043,-0.255743
380,2.001404,1.700110,-0.156594,0.256757,1.643187,-0.090934,0.045411,0.247198,-0.264666,0.195428,0.290898


In [4]:
#特征选择(相关性分析):采取互信息量法进行计算
from sklearn.feature_selection import mutual_info_classif as MIC
result=MIC(x,y)
result

array([0.12443659, 0.12214758, 0.33152243, 0.33823194, 0.        ,
       0.05266316, 0.04626691, 0.        , 0.19070876, 0.2129458 ,
       0.24480482])

In [5]:
#基于F检验
from sklearn.feature_selection import f_classif
F, pvalues_f = f_classif(x,y)
F

array([114.48893299,  89.91887118, 196.04343095, 260.02793981,
         0.31974296,  12.63658035,  11.26965535,   1.65874761,
       148.13900805,  98.62392381, 129.28208479])

In [6]:
#暂时筛选出5678列不要
x_selected=x[x.columns[[0,2,3,8,9,10]]]

In [37]:
x_selected

,患者年龄,迈瑞CA125,迈瑞HE4,迈瑞IGF-1,迈瑞CA153,迈瑞CYFRA 21-1
0,-1.123404,0.447869,0.152925,-1.966062,-0.231347,1.944419
1,0.682041,0.139876,0.837184,-1.885958,0.071807,0.682521
2,-0.567882,-0.287742,-0.367725,-0.234867,-0.185615,-0.325093
3,0.473720,0.313754,1.071110,-1.376661,-0.205168,2.454345
4,0.682041,0.089744,4.685593,-1.515881,0.549162,0.010779
...,...,...,...,...,...,...
377,1.029241,-0.343898,-0.354663,-0.806004,-0.157982,-0.186393
378,1.168122,5.682950,4.685593,-1.770449,0.113499,1.993371
379,0.959801,-0.338344,-0.363924,0.118228,-0.156043,-0.255743
380,2.001404,-0.156594,0.256757,-0.264666,0.195428,0.290898


In [8]:
#[114.48893299,   196.04343095, 260.02793981,148.13900805,  98.62392381, 129.28208479]
#'患者年龄'     '迈瑞CA125'   '迈瑞HE4'   '迈瑞IGF-1', '迈瑞CA153', '迈瑞CYFRA 21-1'

In [9]:
#进行SVM分类计算(寻找最优SVM)
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
svm_model=SVC(kernel='sigmoid')
param_grid = {'C' : np.arange(0.01,1,0.01),
              'gamma':np.arange(0.01,0.5,0.01)}
clf=GridSearchCV(svm_model,param_grid=param_grid,cv=5,n_jobs=-1,scoring='accuracy',verbose=1)
clf.fit(x_selected,y)
#最优参数为{'C': 0.1, 'gamma': 0.06}

Fitting 5 folds for each of 4851 candidates, totalling 24255 fits


GridSearchCV(cv=5, estimator=SVC(kernel='sigmoid'), n_jobs=-1,
             param_grid={'C': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0....
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99]),
                         'gamma': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49])},
             scoring='accura

In [10]:
clf.best_params_

{'C': 0.91, 'gamma': 0.23}

In [11]:
clf.best_score_
#绘制迭代曲线

0.9477443609022556

In [57]:
svm_model=SVC(kernel='sigmoid',C=0.91,gamma=0.23)
x_train,x_test,y_train,y_test=train_test_split(x_selected,y,test_size=0.3,random_state=10)
svm_model.fit(x_train,y_train)
train_pred=svm_model.predict(x_train)
train_acc = accuracy_score(y_train,train_pred)
print('Train Accuracy: ', train_acc)
test_pred =svm_model.predict(x_test)
test_acc = accuracy_score(y_test,test_pred)
print('Test Accuracy:', test_acc)

Train Accuracy:  0.9288389513108615
Test Accuracy: 0.9478260869565217


In [58]:
predict__=svm_model.predict(x_selected)
for i in range(len(predict__)):
    if predict__[i]!=y[i]:
        print(i+1,end=' ')

3 8 23 34 44 120 136 174 181 193 285 289 292 294 295 296 299 303 304 319 337 359 360 365 369 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
model1_log=LogisticRegression(penalty='l2')
param_grid={
    'max_iter' : np.arange(100,1001,100),
    'C' : np.arange(0.1,5,0.1)
}
clf=GridSearchCV(model1_log,param_grid=param_grid,cv=5,n_jobs=-1,scoring='accuracy',verbose=1)
clf.fit(x_selected,y)

In [59]:
#逻辑回归进行二分类
from sklearn.linear_model import LogisticRegression
model_log = LogisticRegression(penalty='l2',max_iter=1000,C=1.0)
x_train,x_test,y_train,y_test=train_test_split(x_selected,y,test_size=0.3,random_state=10)
model_log.fit(x_train,y_train)
train_pred=model_log.predict(x_train)
train_acc = accuracy_score(y_train,train_pred)
print('Train Accuracy: ', train_acc)
test_pred = model_log.predict(x_test)
test_acc = accuracy_score(y_test,test_pred)
print('Test Accuracy:', test_acc)

Train Accuracy:  0.9325842696629213
Test Accuracy: 0.9652173913043478


In [ ]:
predict__=model_log.predict(x_selected)
for i in range(len(predict__)):
    if predict__[i]!=y[i]:
        print(i+1,end=' ')

In [13]:
from sklearn.model_selection import GridSearchCV
model1_log=LogisticRegression(penalty='l2')
max_iter=np.arange(100,1001,100)
C=np.arange(0.1,5,0.1)
param_grid={
    'max_iter' : np.arange(100,1001,100),
    'C' : np.arange(0.1,5,0.1)
}
clf=GridSearchCV(model1_log,param_grid=param_grid,cv=5,n_jobs=-1,scoring='accuracy',verbose=1)
clf.fit(x_selected,y)

Fitting 5 folds for each of 490 candidates, totalling 2450 fits


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
       1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
       2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
       4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9]),
                         'max_iter': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000])},
             scoring='accuracy', verbose=1)

In [48]:

from sklearn.ensemble import RandomForestClassifier
param_grid = {
        "n_estimators": [50,100,150,200,300], # 多少棵树
        "max_depth": [3,4,5,6,7], # 树的最大深度
    }
rd_model=RandomForestClassifier()
clf_rd = GridSearchCV(rd_model, param_grid, cv=5, scoring='accuracy',verbose=1,n_jobs=-1)
clf_rd.fit(x_selected,y)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'n_estimators': [50, 100, 150, 200, 300]},
             scoring='accuracy', verbose=1)

In [49]:
clf_rd.best_params_

{'max_depth': 3, 'n_estimators': 150}

In [50]:
clf_rd.best_score_

0.9318865345181134

In [51]:
forest1 = RandomForestClassifier(n_estimators=150, random_state=0, n_jobs=-1,max_depth=3)
forest1.fit(x_train, y_train)
score = forest1.score(x_test, y_test)
score

0.9652173913043478

In [52]:
#判决指标的重要性
feat_labels=x_selected.columns
importances = forest1.feature_importances_
indices = np.argsort(importances)[::-1] # 下标排序
for f in range(x_selected.shape[1]):   # x_train.shape[1]=7
    print("%2d) %-*s %f" % \
          (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) 迈瑞CA125                        0.402139
 2) 迈瑞HE4                          0.327614
 3) 迈瑞CA153                        0.101979
 4) 迈瑞CYFRA 21-1                   0.092679
 5) 迈瑞IGF-1                        0.041797
 6) 患者年龄                           0.033792


In [53]:
#运用GBDT树
from xgboost import XGBClassifier
param_grid = {
        "n_estimators": [50,100,150,200,300], # 多少棵树
        "learning_rate": [0.05, 0.1, 0.2, 0.3], # 学习率
        "max_depth": [3,4,5,6,7], # 树的最大深度
        "subsample": [0.4,0.6,0.8,1], # 选择多少列构建一个树
    }
xg_model=XGBClassifier()
clf_xgb = GridSearchCV(xg_model, param_grid, cv=5, scoring='accuracy',verbose=1,n_jobs=-1)
clf_xgb.fit(x_selected,y)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                        

In [54]:
clf_xgb.best_params_

{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.4}

In [55]:
gbdt_model=XGBClassifier(learning_rate=0.05, max_depth=3, n_estimators=50, subsample= 0.4)
gbdt_model.fit(x_train, y_train)
print(gbdt_model.score(x_test, y_test))

0.9739130434782609


In [56]:
importances = gbdt_model.feature_importances_
indices = np.argsort(importances)[::-1] # 下标排序
for f in range(x_train.shape[1]):   # x_train.shape[1]=7
    print("%2d) %-*s %f" % \
          (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) 迈瑞HE4                          0.411135
 2) 迈瑞CA125                        0.336622
 3) 迈瑞CA153                        0.082996
 4) 迈瑞IGF-1                        0.071358
 5) 患者年龄                           0.051021
 6) 迈瑞CYFRA 21-1                   0.046868
